# **Практическая работа 2 + Домашняя работа**

In [ ]:
import torch
import math
from torch.autograd import Function
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### **Задание 1**: реализуйте backward для Polynomial 0.5 * (5 * input ** 3 - 3 * input)

In [ ]:
class Polynomial(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return 0.5 * (5 * input ** 3 - 3 * input)

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        return grad_output * (15/2 * (input**2) - 3/2)

In [ ]:
def show_tensor_params(*tensors):
  for x in tensors:
    print('---')
    print(f"data - {x.data}")
    print(f"grad - {x.grad}")
    print(f"grad_fn - {x.grad_fn}")
    print(f"req_grad - {x.requires_grad}")
    print(f"is_leaf - {x.is_leaf}")

In [ ]:
a = float(5)
print(0.5 * (5 * a ** 3 - 3 * a))

305.0


In [ ]:
dtype = torch.float
device = torch.device("cuda")

x1 = torch.tensor(a, requires_grad=True)
output = Polynomial.apply(x1)
output
output.backward()
show_tensor_params(output)
show_tensor_params(x1)

---
data - 305.0
grad - None
grad_fn - <torch.autograd.function.PolynomialBackward object at 0x7f8e0d0bad60>
req_grad - True
is_leaf - False
---
data - 5.0
grad - 186.0
grad_fn - None
req_grad - True
is_leaf - True


<ipython-input-3-168b1ceafb31>:5: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:480.)
  print(f"grad - {x.grad}")


### **Задание 2.** Написать собственный движок автоматического дифференцирования

In [ ]:
class Value:
    # конструктор класса value
    def __init__(self, data, _children=(), _op=''):
        # хранение текущих данных и градиента для backward
        self.data = data
        self.grad = 0
        # внутренние переменные, используемые для построения графа autograd
        self._backward = lambda: None # функция 
        self._prev = set(_children) # набор объектов Value
        self._op = _op # операция, создавшая этот узел, строка ('+', '-', и т д)

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+') # интерпретируем операцию сложения

        def _backward():
            self.grad += out.grad # производная единица
            other.grad += out.grad # аналогично
        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*') # аналогично с умножением

        def _backward():
            self.grad += other.data * out.grad # производная равна other.data
            other.grad += self.data * out.grad # аналогично с self.data
        out._backward = _backward

        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}') # возводим в степень

        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad # берем производную
        out._backward = _backward

        return out

    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward

        return out

    def backward(self):

        # topological order all of the children in the graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1
        for v in reversed(topo):
            v._backward()

    def __neg__(self): # -self
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

Тестирование 

In [ ]:
def test_sanity_check():

    x = Value(-4.0)
    z = 2 * x + 2 + x
  
    q = z.relu() + z * x
    h = (z * z).relu()
    y = h + q + q * x
    y.backward()
    xmg, ymg = x, y

    x = torch.Tensor([-4.0]).double()
    x.requires_grad = True
    z = 2 * x + 2 + x
    q = z.relu() + z * x
    h = (z * z).relu()
    y = h + q + q * x
    y.backward()
    xpt, ypt = x, y

    
    # forward pass went well
    assert ymg.data == ypt.data.item()
    # backward pass went well
    print(xmg, xpt, xpt.grad)
    assert xmg.grad == xpt.grad.item()

In [ ]:
def test_more_ops():

    a = Value(-4.0)
    b = Value(2.0)
    c = a + b
    d = a * b + b**3
    c += c + 1
    c += 1 + c + (-a)
    d += d * 2 + (b + a).relu()
    d += 3 * d + (b - a).relu()
    e = c - d
    f = e**2
    g = f / 2.0
    g += 10.0 / f
    g.backward()
    amg, bmg, gmg = a, b, g

    a = torch.Tensor([-4.0]).double()
    b = torch.Tensor([2.0]).double()
    a.requires_grad = True
    b.requires_grad = True
    c = a + b
    d = a * b + b**3
    c = c + c + 1
    c = c + 1 + c + (-a)
    d = d + d * 2 + (b + a).relu()
    d = d + 3 * d + (b - a).relu()
    e = c - d
    f = e**2
    g = f / 2.0
    g = g + 10.0 / f
    g.backward()
    apt, bpt, gpt = a, b, g

    tol = 1e-6
    # forward pass went well
    assert abs(gmg.data - gpt.data.item()) < tol
    # backward pass went well
    assert abs(amg.grad - apt.grad.item()) < tol
    assert abs(bmg.grad - bpt.grad.item()) < tol

In [ ]:
a = Value(-4.0)
b = Value(2.0)
d = Value(3.0)

In [ ]:
c = a + b
e = c * d
e.backward()

In [ ]:
test_sanity_check()

Value(data=-4.0, grad=46.0) tensor([-4.], dtype=torch.float64, requires_grad=True) tensor([46.], dtype=torch.float64)


In [ ]:
test_more_ops()

### **Обучение на основе собственной библиотеки** (WIP) - надо доделать

#### *Многослойный перцептрон на основе класса Value*

In [ ]:
import random

class Module:

    def zero_grad(self):
        for tmp in self.parameters():
            tmp.grad = 0

    def parameters(self):
        return []

class Neuron(Module):

    def __init__(self, nin, nonlin=True):
        self.w = ...
        self.b = Value(0)
        self.nonlin = nonlin

    def __call__(self, x):
        act = ...
        return act.relu() if self.nonlin else act

    def parameters(self):
        return self.w + [self.b]

    def __repr__(self):
        return f"{'ReLU' if self.nonlin else 'Linear'}Neuron({len(self.w)})"

class Layer(Module):

    def __init__(self, nin, nout, **kwargs):
        self.neurons = ...

    def __call__(self, x):
        out = [n(x) for n in self.neurons]
        return out[0] if len(out) == 1 else out

    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]

    def __repr__(self):
        return f"Layer of [{', '.join(str(n) for n in self.neurons)}]"

class MLP(Module):

    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1], nonlin=(i!=len(nouts)-1)) for i in range(len(nouts))]
        
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

    def __repr__(self):
        repr = '\n'.join(str(layer) for layer in self.layers)
        return f"MLP of [{repr}]"


#### *Обучение многослойного перцептрона*



Сам перцептрон

In [ ]:
nn = MLP(3, [4, 4, 1])
print(model)
print("number of parameters", len(model.parameters()))

Набор данных

In [ ]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets

In [ ]:
for k in range(20):

    # forward
    ...

    # calculate loss (mean square error)
    ...
    
    # backward (zero_grad + backward)
    ...
    
    # update
    learning_rate = ...
    for p in n.parameters():
        ...
    
    if k % 1 == 0:
        print(f"step {k} loss {total_loss.data}, accuracy {acc*100}%")